读取Excel 内容

In [ ]:
import os
import pandas as pd

# 指定要读取的文件夹路径
folder_path = r'D:\lxd\learn\py\tvbox'

# 遍历文件夹中的.xlsx文件
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):  # 确保文件是Excel文件
        file_path = os.path.join(folder_path, filename)
        
        # 使用pandas读取Excel文件
        try:
            df = pd.read_excel(file_path)
            print(f"正在处理文件: {filename}")
            # 在这里处理df，比如打印内容、分析数据等
            print(df.head())  # 打印每份文件的前几行作为示例
        except Exception as e:
            print(f"读取文件{filename}时出错: {e}")

In [ ]:
   import pandas as pd

   # 读取单个sheet
   excel_file = r'D:\lxd\learn\py\tvbox\excel_xianlu.xlsx'
   df = pd.read_excel(excel_file)
   print(df)

   # 读取特定sheet
   df_sheet1 = pd.read_excel(excel_file, sheet_name='Sheet1')
   print(df_sheet1)

批量读取第二列的urls数据

In [3]:
from openpyxl import load_workbook
excel_file = r'D:\lxd\learn\py\tvbox\excel_xianlu.xlsx'
workbook = load_workbook(filename=excel_file)
# sheet = workbook.active  # 获取活动工作表
sheet = workbook['Sheet1']  # 根据名称获取工作表

for row in sheet.iter_rows(values_only=True):
    # print(row)
    print(row[1],type(row[1]))
    

http://154.3.2.66:8889/down/YgOQS9WjkF6R.txt <class 'str'>
http://101.200.122.188/666 <class 'str'>
http://39.101.135.137:8080/ <class 'str'>
http://39.101.135.137:8686 <class 'str'>
http://39.101.135.137:8686/ <class 'str'>
http://52bsj.vip:98/cl <class 'str'>
http://52bsj.vip:98/wuai <class 'str'>
http://52bsj.vip:98/wuaihouse <class 'str'>
http://52pan.top:81/api/v3/file/get/174964/%E5%90%BE%E7%88%B1%E8%AF%84%E6%B5%8B.m3u?sign=rPssLoffquDXszCARt6UNF8MobSa1FA27XomzOluJBY%3D%3A0 <class 'str'>
http://52pan.top:81/api/v3/file/get/174964/吾爱评测.m3u?sign=rPssLoffquDXszCARt6UNF8MobSa1FA27XomzOluJBY%3D%3A0 <class 'str'>
http://530.my3v.work/99668.json <class 'str'>
http://530.my3v.work/hxck.json <class 'str'>
http://530.my3v.work/qm.json <class 'str'>
http://530.my3v.work/xdck.json <class 'str'>
http://8.210.232.168/xc.json <class 'str'>
http://8.210.232.168/xclive.txt <class 'str'>
http://9xi4o.tk/0725.json <class 'str'>
http://bp.tvbox.cam <class 'str'>
http://cdn.qiaoji8.com/tvbox.json <cl

20240515 pm excel 批量读取URL后识别线路 并输出

In [2]:
import openpyxl
import requests
import json
import re,os

def refindvalue(respon): #输入respond,re正则判断单仓多仓，返回cang自定义文本
    #mcang=re.search('storeHouse',respon,re.M|re.I).group()
    #scang = re.search('urls',respon,re.M|re.I).group()
    #xlu = re.search('sites',respon,re.M|re.I).group()
    try: #正常的操作
        mcang = re.search('sites',respon,re.M|re.I).group()
    except :#发生异常，执行这块代码
        try:
            scang = re.search('storeHouse',respon,re.M|re.I).group()
        except:#发生异常，执行这块代码
            try:
                xlu = re.search('urls',respon,re.M|re.I).group()
            except:#发生异常，执行这块代码
                return "正则失败"
            else: #如果没有异常执行这块代码
                return "H单仓"
        else:
            return "H多仓"
    else:
        return "H线路"
    
def addlink(url): #增加代理链接
    agentlink="http://fly.lyin.cf/?url="
    agentlink1="http://lige.unaux.com/?url="
    agentlink0=""
    urll=agentlink1+url
    # print("代理网址： "+urll)
    return urll

def re_findall(url): #输入原始url,解析网站，返回url，respone等元组
    headers={
    "Cookie":"Hm_lvt_d815f2e1f682c86565c1063aafaef292=1715176162; Hm_lpvt_d815f2e1f682c86565c1063aafaef292=1715176162; tfstk=fxg2k1gSVEL2a_igU8aaUOQXoJ4YWypCbVw_sfV4e-4scq_Z4XMWMi_ssRkZF8HaXIe_IRkssmOWReGxMPUg7pTBRYdKSDg4IOagkckz-p9BRUGxMPUMdc6Dftlz68fcjoDgru281Z4gnlXlqW2uIP4iI74uTSfcmZqGZ7DZTXhNG5ZDHRmfw4bLXoyngkQG747Q0Jc0UNV-zS7gKj2PSNwY5nQEaj9VJbaxZvPIhUbqLbGIku3wLErxrXu80VCN2bZL3z2xcK78gxGZ8u0D16wmyf0L4y-Od83tZDZL-tj0Try4xqD5sB40j540Hm9VySkj_2rooCsQwcD-7raHJt2sVXua_Vp9zvlSOqE-7p8l4NBTZ0ZxWmWGboV8aJOyZMO1RXe6S7PCXGEYo7yBizIGj9QN5iu6ZGITDWVzdI4O.; __test=1af893b932aa21ad543a801402580fba",
    "Host":"lige.unaux.com",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Connection":"keep-alive"
    }
    proxy_url=addlink(url)  #增加代理函数
    try:
        respon=requests.get(url)
        codema= respon.status_code
        if codema==200:
            # print("首次解析的网址返回状态： "+str(codema)+"- "+url)
            conten="首次解析状态:"+str(codema)
            return url,respon.text,conten
        else:
            respon1=requests.get(proxy_url,headers=headers)  #增加代理1函数
            codema1= respon1.status_code
            # print("代理1网址返回状态： "+str(codema)+"- "+proxy_url)
            conten1="代理1网址状态: "+str(codema1)
            return proxy_url,respon1.text,conten1
    except:
        try:
            respon2=requests.get(proxy_url,headers=headers)  #增加代理2函数
            codema2= respon2.status_code,conten
            # print("代理2网址返回状态： "+str(codema)+"- "+proxy_url)
            conten2="代理2网址状态: "+str(codema2)
            return proxy_url,respon2.text,conten2
        except:
            # print("解析失败： "+url)
            return url,"000","解析失败了: "


def jiexi(urll):
        analysis_respone=re_findall(urll) #返回元组，第一个是url，第二个是respon，第三个是状态内容
        # cang=findvalue(analysis_respone[1])
        cang=refindvalue(analysis_respone[1])
        # print("shuchu--",cang,analysis_respone)

        if cang=="H单仓":
            print(analysis_respone[2]+" 单仓"+"-"+analysis_respone[0])
        elif cang=="H多仓":
            print(analysis_respone[2] +" 多仓"+"-"+analysis_respone[0])
        elif cang=="H线路":
            print(analysis_respone[2]+" 线路"+"-"+analysis_respone[0])
        else:
            # pass
            print(analysis_respone[2]+" 其他failure "+"-"+analysis_respone[0])
        # print("="*30)

i=1
excel_file = r'D:\lxd\learn\py\tvbox\excel_xianlu.xlsx'
workbook = openpyxl.load_workbook(filename=excel_file)
sheet = workbook['Sheet1']
sheet2= workbook['Sheet2']
for row in sheet2.iter_rows(values_only=True):
    # print(row[1])
    if not (row[1]) is None :
        analysis_respone=re_findall(row[1]) 
        strtxt=refindvalue(analysis_respone[1])
        sheet2.cell(row = i,column = 3,value = strtxt)
        
        print(row[1],i,strtxt)
    else:
        break
    i=i+1
workbook.save(filename = "excel_t1.xlsx")

https://raw.githubusercontent.com/xiqiliuse/tvbox/main/tvbox.json 1 正则失败
http://gh.whoneil.top/https://raw.githubusercontent.com/xiqiliuse/tvbox/main/tvbox.json 2 H单仓
http://39.101.135.137:8080/ 3 H线路
http://39.101.135.137:8686 4 正则失败
http://39.101.135.137:8686/ 5 正则失败
http://52bsj.vip:98/cl 6 正则失败
http://52bsj.vip:98/wuai 7 正则失败
http://52bsj.vip:98/wuaihouse 8 正则失败


In [ ]:
import openpyxl
import requests
import json
import re,os
import warnings 

warnings.filterwarnings("ignore")

def refindvalue(respon): #输入respond,re正则判断单仓多仓，返回cang自定义文本
    #mcang=re.search('storeHouse',respon,re.M|re.I).group()
    #scang = re.search('urls',respon,re.M|re.I).group()
    #xlu = re.search('sites',respon,re.M|re.I).group()
    try: #正常的操作
        mcang = re.search('sites',respon,re.M|re.I).group()
    except :#发生异常，执行这块代码
        try:
            scang = re.search('storeHouse',respon,re.M|re.I).group()
        except:#发生异常，执行这块代码
            try:
                xlu = re.search('urls',respon,re.M|re.I).group()
            except:#发生异常，执行这块代码
                return "正则失败"
            else: #如果没有异常执行这块代码
                return "H单仓"
        else:
            return "H多仓"
    else:
        return "H线路"
    
def addlink(url): #增加代理链接
    agentlink="http://fly.lyin.cf/?url="
    agentlink1="http://lige.unaux.com/?url="
    agentlink0=""
    urll=agentlink1+url
    # print("代理网址： "+urll)
    return urll

def re_findall(url): #输入原始url,解析网站，返回url，respone等元组
    headers={
    "Cookie":"Hm_lvt_d815f2e1f682c86565c1063aafaef292=1715176162; Hm_lpvt_d815f2e1f682c86565c1063aafaef292=1715176162; tfstk=fxg2k1gSVEL2a_igU8aaUOQXoJ4YWypCbVw_sfV4e-4scq_Z4XMWMi_ssRkZF8HaXIe_IRkssmOWReGxMPUg7pTBRYdKSDg4IOagkckz-p9BRUGxMPUMdc6Dftlz68fcjoDgru281Z4gnlXlqW2uIP4iI74uTSfcmZqGZ7DZTXhNG5ZDHRmfw4bLXoyngkQG747Q0Jc0UNV-zS7gKj2PSNwY5nQEaj9VJbaxZvPIhUbqLbGIku3wLErxrXu80VCN2bZL3z2xcK78gxGZ8u0D16wmyf0L4y-Od83tZDZL-tj0Try4xqD5sB40j540Hm9VySkj_2rooCsQwcD-7raHJt2sVXua_Vp9zvlSOqE-7p8l4NBTZ0ZxWmWGboV8aJOyZMO1RXe6S7PCXGEYo7yBizIGj9QN5iu6ZGITDWVzdI4O.; __test=1af893b932aa21ad543a801402580fba",
    "Host":"lige.unaux.com",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Connection":"keep-alive"
    }
    proxy_url=addlink(url)  #增加代理函数
    try:
        respon=requests.get(url,verify=False,timeout =60)
        codema= respon.status_code
        if codema==200:
            # print("首次解析的网址返回状态： "+str(codema)+"- "+url)
            conten="首次解析状态:"+str(codema)
            return url,respon.text,conten
        else:
            respon1=requests.get(proxy_url,headers=headers,verify=False,timeout =60)  #增加代理1函数
            codema1= respon1.status_code
            # print("代理1网址返回状态： "+str(codema)+"- "+proxy_url)
            conten1="代理1网址状态: "+str(codema1)
            return proxy_url,respon1.text,conten1
    except:
        try:
            respon2=requests.get(proxy_url,headers=headers,verify=False,timeout =60)  #增加代理2函数
            codema2= respon2.status_code,conten
            # print("代理2网址返回状态： "+str(codema)+"- "+proxy_url)
            conten2="代理2网址状态: "+str(codema2)
            return proxy_url,respon2.text,conten2
        except:
            # print("解析失败： "+url)
            return url,"000","解析失败了: "


def jiexi(urll):
        analysis_respone=re_findall(urll) #返回元组，第一个是url，第二个是respon，第三个是状态内容
        # cang=findvalue(analysis_respone[1])
        cang=refindvalue(analysis_respone[1])
        # print("shuchu--",cang,analysis_respone)

        if cang=="H单仓":
            print(analysis_respone[2]+" 单仓"+"-"+analysis_respone[0])
        elif cang=="H多仓":
            print(analysis_respone[2] +" 多仓"+"-"+analysis_respone[0])
        elif cang=="H线路":
            print(analysis_respone[2]+" 线路"+"-"+analysis_respone[0])
        else:
            # pass
            print(analysis_respone[2]+" 其他failure "+"-"+analysis_respone[0])
        # print("="*30)

i=1
# excel_file = os.getcwd()+"/excel_xianlu.xlsx"
excel_file = r'D:\lxd\learn\py\tvbox\excel_xianlu.xlsx'
workbook = openpyxl.load_workbook(filename=excel_file)
# sheet = workbook['Sheet1']
sheet2= workbook['Sheet2']
for row in sheet2.iter_rows(values_only=True):
    # print(row[1])
    if not (row[1]) is None :
        analysis_respone=re_findall(row[1]) 
        strtxt=refindvalue(analysis_respone[2])
        codetxt=analysis_respone[2]
        sheet2.cell(row = i,column = 5,value = strtxt)
        sheet2.cell(row = i,column = 6,value = codetxt)
        print(row[1],i,strtxt,codetxt)
    else:
        break
    i=i+1
workbook.save(filename = "excel_0517pm.xlsx")